# 04. 🌐 Standard Library Reference: `socket` (Networking)

The **`socket`** module is the bedrock of Python networking. It provides access to the low-level network interface of your operating system.

### 🧐 The "Telephone" Analogy
Think of a **socket** like a **telephone endpoint**.
* **The Socket:** The phone itself.
* **IP Address:** The phone number you are calling.
* **Port:** The specific extension number (e.g., Ext. 80 for Web, Ext. 443 for Secure Web).



**Key Topics Covered:**
* **Concept:** Understanding `AF_INET` and `SOCK_STREAM`.
* **Protocols:** TCP (Reliable) vs. UDP (Fast).
* **The Flow:** Connect $\rightarrow$ Send $\rightarrow$ Receive.
* **Utility:** Finding IP addresses and hostnames.

In [1]:
import socket
import time

## 1.1 🧠 Core Concepts: TCP vs UDP

When you create a socket, you have to decide **how** you want to communicate. This is done by passing two arguments to `socket.socket()`.

### 1. The Address Family (`AF_INET`)
This tells Python we are using **IPv4** addresses (like `192.168.1.1`).

### 2. The Socket Type (`SOCK_STREAM` vs `SOCK_DGRAM`)

| Protocol | Type | Reliability | Analogy | Use Case |
| :--- | :--- | :--- | :--- | :--- |
| **TCP** | `SOCK_STREAM` | **High** (Guaranteed Order) | A Phone Call (Connection-based) | Web (HTTP), Email, SSH |
| **UDP** | `SOCK_DGRAM` | **Low** (Fire and Forget) | Sending a Letter (No guarantee) | Streaming Video, Gaming, DNS |




In [2]:
# Create a standard TCP/IPv4 socket (The most common type)
# AF_INET = IPv4
# SOCK_STREAM = TCP
tcp_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
print(f"TCP Socket Type: {tcp_socket.type}")

# Create a UDP/IPv4 socket
# SOCK_DGRAM = UDP
udp_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
print(f"UDP Socket Type: {udp_socket.type}")

## 1.2 📞 Client Setup (The "Phone Call")

To retrieve data from a server, we follow a strict sequence of steps. This is the exact code that runs "under the hood" when you use a web browser.

1.  **`socket()`**: Pick up the phone.
2.  **`connect()`**: Dial the number (Host and Port).
3.  **`send()`**: Speak your request (must be in **bytes**).
4.  **`recv()`**: Listen for the answer (comes back in **bytes**).
5.  **`close()`**: Hang up.

**Note on Encoding:** The internet speaks in `bytes`, not Python strings. We use `.encode('utf-8')` to send and `.decode('utf-8')` to read.

In [ ]:
HOST = 'www.google.com'
PORT = 80 # Standard HTTP port (The "Door" for web traffic)

# We use 'with' to ensure the socket closes automatically (Hangs up the phone)
try:
    print(f"Dialing {HOST} on port {PORT}...")
    with socket.create_connection((HOST, PORT), timeout=5) as client_socket:
        
        # 1. Prepare the Request
        # This is a raw HTTP GET request. Note the \r\n newlines required by the protocol.
        request = 'GET / HTTP/1.1\r\nHost: www.google.com\r\nConnection: close\r\n\r\n'
        
        # 2. Send (Speak)
        client_socket.sendall(request.encode('utf-8'))

        # 3. Receive (Listen)
        # We listen for up to 4096 bytes of data
        response = client_socket.recv(4096)
        
        print(f"\n--- 📩 HTTP Response (First 100 chars) ---")
        print(response.decode('utf-8')[:100])
        
except Exception as e:
    print(f"Could not connect: {e}")

## 1.3 🧑‍💻 Network Utilities

Sometimes you need to know "Who am I?" or "Where is GitHub?" on the network. The socket library includes DNS (Domain Name System) lookups.



In [7]:
print(f"Local Hostname: {socket.gethostname()}")

# Resolving a hostname to an IP address (DNS Lookup)
# This converts 'www.github.com' -> '20.205.243.166'
try:
    ip_address = socket.gethostbyname('www.github.com')
    print(f"GitHub IP: {ip_address}")
except socket.gaierror:
    print("Could not resolve hostname.")

# Getting service name from port
# Port 22 is usually SSH, 80 is HTTP, 443 is HTTPS
service_name = socket.getservbyport(22)
print(f"Port 22 Service: {service_name}")

---

## 潮 Mini-Challenge: The Basic Echo Server (Conceptual)

**Task:** Write the conceptual steps for a single-threaded TCP server that echoes back any data it receives.

**The Server Logic:**
Unlike a client (which dials), a server must **wait** for a call.
1.  **Bind:** Attach the socket to a specific address/port (Claim the phone number).
2.  **Listen:** Wait for the phone to ring.
3.  **Accept:** Pick up the phone when it rings (creates a *new* socket for that specific caller).
4.  **Recv/Send:** Talk.



In [ ]:
HOST = '127.0.0.1' # Localhost (Your own computer)
PORT = 8080        # A random open port

# Conceptual Server Logic (Cannot run in notebook as it blocks)
try:
    # 1. Create
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    # 2. Bind
    server_socket.bind((HOST, PORT))
    
    # 3. Listen
    server_socket.listen(1) # Queue up to 1 connection
    print(f"Server listening on {HOST}:{PORT}...")

    # 4. Accept (BLOCKING CALL - The code stops here until someone connects)
    # conn = The specific connection to the client
    # addr = The IP address of the client
    conn, addr = server_socket.accept()
    
    with conn:
        print(f"Connected by {addr}")
        while True:
            data = conn.recv(1024)
            if not data: break
            # Echo back the data
            conn.sendall(b'Echo: ' + data)

except Exception as e:
    print("Conceptual code structure complete.")

---

## 🌟 Core Insight for Your CSE Career

### Sockets are the Foundation
Every time you use `requests.get()`, or load a page in Chrome, or use FastAPI, **this code is running underneath**.

* **FastAPI** handles thousands of these socket connections for you using `async` loops.
* **Web Browsers** manage the complexity of rendering the HTML that comes back from the `recv()` call.

Understanding sockets means you understand the **Transport Layer** of the internet.